<h2>Script: Robô automação siscori</h2>
<h4>Autor: Jonathan G </h4>
<h4>Data: 26/06/2020</h4>

In [1]:
from selenium import webdriver
import urllib.request
from selenium.webdriver.common.keys import Keys
from anticaptchaofficial.imagecaptcha import *
from selenium.webdriver.firefox.options import Options
import datetime
import os
from selenium.common.exceptions import NoSuchElementException

In [2]:
#Declarando tipo do download 
tpDownImp = 'CAPI' # IMPORTAÇÃO
tpDownExp = 'CAPE' # EXPORTAÇÃO
#Declarando os botões de Imp e Exp da tela
clickImp = '#principal:j_idt45:j_idt46'
clickExp = '#principal:j_idt45:j_idt62'
#Lista de arquivos que não foram localizados
listaNaoLoc = []
#Declando o número de arquivos que devem ser feito download
cont = 1


In [3]:
#Pegando o nome do mês atual(3 letras), ano e mês de download. Subtraindo 10 dias para pegar o mês anterior
mydate = datetime.datetime.now() - datetime.timedelta(days = 10)
mesAtual = mydate.strftime("%b")
anoAtual = mydate.year
mesDown =  mydate.month

In [6]:
#caminho de download
path = "/home/jonathan/Documentos/SEARCHX/"
pathMes = path+str(anoAtual)+"/"+mesAtual
pathImp = pathMes+"/Imp"
pathExp = pathMes+"/Exp"

#Cria as pastas caso não existam
if (os.path.isdir(pathMes)):
    print("Pasta já foi criada!")
else:
    os.mkdir(pathMes)
    os.mkdir(pathImp)
    os.mkdir(pathExp)
    print("Pastas não existem, mas foram criadas")

Pastas não existem, mas foram criadas


In [7]:
def iniciarBot(caminho,tpDown,tpClick,cont):
    #Toda vez que é chamada a função webdriver.Firefox, uma nova instância é criada e um novo perfil
    #Aqui embaixo é feita a configuração do perfil para que o download aconteça de forma automática, sem a janela de popUp
    profile = webdriver.FirefoxProfile()
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.download.dir", caminho)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/zip")

    #Chama driver do firefox, não usei chrome porque da problema de certifcado com o site
    #Acessa o site e procura pelo id e pega o atributo src que é a imagem do captcha
    driver = webdriver.Firefox(firefox_profile=profile)
    driver.get("http://siscori.receita.fazenda.gov.br/apoiosiscori/consulta.jsf")
    
    rotinaDownload(tpDown,driver,tpClick,caminho,cont)

In [8]:
def rotinaDownload(tpDown,driver,tpClick,caminho,cont):
    
    
    while (cont < 98):
        imagemCaptcha = driver.find_element_by_id("img_captcha_serpro_gov_br")

        #Necessário executar depois que a página renderiza

        time.sleep(5)
        imagemCaptcha = imagemCaptcha.get_attribute("src")

        #Faz uma requisição no link da imagem e baixa na pasta especificada que fica junto com esse código
        urllib.request.urlretrieve(imagemCaptcha, "Captchas/cp1.png")

        #Chamando anticaptcha
        #chave 
        solver = imagecaptcha()
        solver.set_verbose(1)
        solver.set_key("")

        captcha_text = solver.solve_and_return_solution("Captchas/cp1.png")
        if captcha_text != 0:
            print("captcha text "+captcha_text)
        else:
            print("task finished with error "+solver.error_code)
        
        #Aqui ele vai tentar localizar o arquivo no html, se ele não achar significa que naquele mês não teve importação do produto
        try:
            #Selecionando o campo do Captcha e setando o resultado retornado
            campoCaptcha = driver.find_element_by_name("txtTexto_captcha_serpro_gov_br")
            campoCaptcha.send_keys(captcha_text)

            #Selecionando o campo de arquivo e baixando
            driver.find_element_by_xpath("//*[contains(@href,'"+tpClick+"')]").click()
            time.sleep(1)
            linkDownload = driver.find_element_by_xpath("//*[contains(@title,'%s%02d20%02d.zip')]" % (tpDown,cont,mesDown))
            linkDownload.click()
            print("Fazendo download do arquivo: %s%02d20%02d.zip" % (tpDown,cont,mesDown))
                     
            #Tempo para esperar finalizar download
            time.sleep(3)
            
            if (os.path.isfile(caminho+"/"+'%s%02d20%02d.zip' % (tpDown,cont,mesDown))):
                print("Download do arquivo %s%02d20%02d.zip conferido e ok" % (tpDown,cont,mesDown))
                #Adicionando mais um no contador depois de fazer dowload do arquivo com sucesso
                cont+=1
                print("Contador está em: "+str(cont))
            else:
                print("Captcha do arquivo %s%02d20%02d.zip foi digitado de modo incorreto, realizando nova tentativa!" % (tpDown,cont,mesDown))      
            
        except NoSuchElementException:
            #Como ele não encontrou no html da página o link, ele adiciona mais um e busca novamente
            print("Produto %s%02d20%02d.zip não foi localizado, fazendo download do próximo!" % (tpDown,cont,mesDown))
            #Listando arquivos que não foram localizados
            listaNaoLoc.append('%s%02d20%02d' % (tpDown,cont,mesDown))
            cont+=1
            print("Contador está em: "+str(cont))
            
    driver.close()

In [1]:
iniciarBot(pathImp, tpDownImp, clickImp,cont)
print("Download Importação concluída!")
iniciarBot(pathExp, tpDownExp, clickExp,cont)
print("Download Exportação concluída!")
print("Segue a lista dos arquivos que não foram localizados: "+listaNaoLoc)